# Target Visibility

In [ ]:
# Parameters cell. Set defaults here
ra = 0.
dec = -30.

In [ ]:
import numpy as np
import scipy

from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.time import Time
from datetime import datetime
from astroplan import Observer, FixedTarget
from astroplan.plots import plot_airmass, plot_sky

import matplotlib.pyplot as plt

In [ ]:
print(ra)
print(dec)